# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [8]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import datetime
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [9]:
vac_df = pd.read_csv("../../output_data/cityweather_data.csv")
vac_df.head()

,City,Lat,Lng,Max temp,Temp,Humidity,Cloudiness,Windspeed,Country,Date
0,Busselton,-33.6500,115.3333,71.04,71.04,46,82,17.36,AU,1639712650
1,Saint-Philippe,-21.3585,55.7679,78.48,77.83,77,50,8.93,RE,1639712651
2,Sarandi,-23.4436,-51.8739,80.15,80.15,69,0,11.50,BR,1639712651
3,Vaini,-21.2000,-175.2000,86.16,86.16,89,20,13.80,TO,1639712651
4,Fairbanks,64.8378,-147.7164,5.59,3.13,81,90,0.00,US,1639712651


In [10]:
vac_df.shape

(531, 10)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [11]:
# Store latitude and longitude in locations
vlocations = vac_df[["Lat","Lng"]]

In [12]:
# Store humidity weight values and convert to float
hum = vac_df["Humidity"].astype(float)
hum

0      46.0
1      77.0
2      69.0
3      89.0
4      81.0
       ... 
526    74.0
527    93.0
528    77.0
529    91.0
530    96.0
Name: Humidity, Length: 531, dtype: float64

In [13]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(vlocations, weights=hum, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=5)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

* Narrow down the DataFrame to find your ideal weather condition. For example:
* A max temperature lower than 80 degrees but higher than 70.
* Wind speed less than 10 mph.
* Zero cloudiness.
* Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
* **Note:** Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

In [15]:
Tindex = vac_df[(vac_df['Max temp']>=70) & (vac_df['Max temp']<=80) & (vac_df['Windspeed']<=10) & (vac_df['Cloudiness']<=0)].index
Tindex

Int64Index([108, 203, 342, 368], dtype='int64')

In [16]:
hotel_df = vac_df.drop(Tindex,inplace=False)
hotel_df

,City,Lat,Lng,Max temp,Temp,Humidity,Cloudiness,Windspeed,Country,Date
0,Busselton,-33.6500,115.3333,71.04,71.04,46,82,17.36,AU,1639712650
1,Saint-Philippe,-21.3585,55.7679,78.48,77.83,77,50,8.93,RE,1639712651
2,Sarandi,-23.4436,-51.8739,80.15,80.15,69,0,11.50,BR,1639712651
3,Vaini,-21.2000,-175.2000,86.16,86.16,89,20,13.80,TO,1639712651
4,Fairbanks,64.8378,-147.7164,5.59,3.13,81,90,0.00,US,1639712651
...,...,...,...,...,...,...,...,...,...,...
526,Bintulu,3.1667,113.0333,84.61,83.91,74,20,1.99,MY,1639713349
527,Mendi,-6.1761,143.6513,64.13,64.13,93,84,2.13,PG,1639713349
528,Bacolod City,10.6667,122.9500,83.43,83.43,77,100,11.18,PH,1639713349
529,Whitley Bay,55.0397,-1.4471,39.20,37.94,91,2,4.61,GB,1639713350


# Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [17]:
hotel_df["Hotel Name"] = ""
hotel_df.head(50)

,City,Lat,Lng,Max temp,Temp,Humidity,Cloudiness,Windspeed,Country,Date,Hotel Name
0,Busselton,-33.6500,115.3333,71.04,71.04,46,82,17.36,AU,1639712650,
1,Saint-Philippe,-21.3585,55.7679,78.48,77.83,77,50,8.93,RE,1639712651,
2,Sarandi,-23.4436,-51.8739,80.15,80.15,69,0,11.50,BR,1639712651,
3,Vaini,-21.2000,-175.2000,86.16,86.16,89,20,13.80,TO,1639712651,
4,Fairbanks,64.8378,-147.7164,5.59,3.13,81,90,0.00,US,1639712651,
5,Hermanus,-34.4187,19.2345,57.79,57.79,72,0,26.33,ZA,1639712652,
6,Tazovsky,67.4667,78.7000,-35.12,-35.12,100,82,5.57,RU,1639712652,
7,Tuktoyaktuk,69.4541,-133.0374,-27.40,-27.40,75,20,13.80,CA,1639712652,
8,Carnarvon,-24.8667,113.6333,75.27,75.27,57,30,20.71,AU,1639712510,
9,Tasiilaq,65.6145,-37.6368,35.76,35.76,99,100,4.90,GL,1639712652,


In [18]:
# params dictionary to update each iteration

params = {
    "radius": 5000,
    "types": "hotel",
    "key": g_key
}
               
# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    
        # get lat, lng from df
        lat = row["Lat"]
        lng = row["Lng"]
        city = row["City"]
        # change location each iteration while leaving original params in place
        params["location"] = f"{lat},{lng}"
        
        #print(f"Retrieving nearest Hotel to City {index}: {city}.")
        # Use the search term: "hotel" and our lat/lng
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        response = requests.get(base_url, params=params)
        name_hotel = response.json()
        # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
        try:
            print(f"{index} Closest {city} hotel is {name_hotel['results'][0]['name']}.")
            hotel_df.loc[index, "Hotel Name"] = name_hotel['results'][0]["name"]
        except (KeyError, IndexError):
            print(f"Missing field...skipping {index} {city}")
            pass           

0 Closest Busselton hotel is Busselton.
1 Closest Saint-Philippe hotel is Saint-Philippe.
2 Closest Sarandi hotel is Maringá.
3 Closest Vaini hotel is Nukuhetulu.
4 Closest Fairbanks hotel is Fairbanks.
5 Closest Hermanus hotel is Hermanus.
6 Closest Tazovsky hotel is Tazovskii.
7 Closest Tuktoyaktuk hotel is Tuktoyaktuk.
8 Closest Carnarvon hotel is Carnarvon.
9 Closest Tasiilaq hotel is Tasiilaq.
10 Closest Upernavik hotel is Upernavik.
11 Closest Leningradskiy hotel is Leningradsky.
12 Closest Albany hotel is Voorheesville.
13 Closest Rikitea hotel is Rikitea.
14 Closest Kaeo hotel is Kaeo.
15 Closest Faanui hotel is Vaitape.
16 Closest Tezu hotel is Tezu.
Missing field...skipping 17 Boende
18 Closest Haines Junction hotel is Haines Junction.
19 Closest Hasaki hotel is Kamisu.
20 Closest Araouane hotel is Araouane.
21 Closest Rabaul hotel is Rabaul.
22 Closest Dubrovka hotel is Dubrovka.
23 Closest Arraial do Cabo hotel is Arraial do Cabo.
24 Closest Kapaa hotel is Kapaʻa.
25 Closes

KeyboardInterrupt: 

In [19]:
hotel_df.head(50)

,City,Lat,Lng,Max temp,Temp,Humidity,Cloudiness,Windspeed,Country,Date,Hotel Name
0,Busselton,-33.6500,115.3333,71.04,71.04,46,82,17.36,AU,1639712650,Busselton
1,Saint-Philippe,-21.3585,55.7679,78.48,77.83,77,50,8.93,RE,1639712651,Saint-Philippe
2,Sarandi,-23.4436,-51.8739,80.15,80.15,69,0,11.50,BR,1639712651,Maringá
3,Vaini,-21.2000,-175.2000,86.16,86.16,89,20,13.80,TO,1639712651,Nukuhetulu
4,Fairbanks,64.8378,-147.7164,5.59,3.13,81,90,0.00,US,1639712651,Fairbanks
5,Hermanus,-34.4187,19.2345,57.79,57.79,72,0,26.33,ZA,1639712652,Hermanus
6,Tazovsky,67.4667,78.7000,-35.12,-35.12,100,82,5.57,RU,1639712652,Tazovskii
7,Tuktoyaktuk,69.4541,-133.0374,-27.40,-27.40,75,20,13.80,CA,1639712652,Tuktoyaktuk
8,Carnarvon,-24.8667,113.6333,75.27,75.27,57,30,20.71,AU,1639712510,Carnarvon
9,Tasiilaq,65.6145,-37.6368,35.76,35.76,99,100,4.90,GL,1639712652,Tasiilaq


In [20]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

# Add marker layer on top of heat map
# Display figure

In [21]:
# Create a marker_layer using the hotel_info to fill the info box
fig = gmaps.figure()
markers = gmaps.marker_layer(locations,
    info_box_content=hotel_info)                        
fig.add_layer(markers)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))